# Subhalo Random Forest Classification

## Train Random Forest Classifier and Analyze Performance

This notebook steps through training a random forest to identify disrupted and surviving subhalos in zoom-in simulations. After loading training data containing appropriate features for each disrupted and surviving subhalo, a random forest classifier is tuned using the $\texttt{GridSearchCV}$ function and its performance (e.g., test set accuracy, true/false positive rates, confusion matrix, ROC curve) is analyzed. We provide our trained model $\texttt{finalized}\_\texttt{rf.sav}$ in the repository.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns 
sns.set_style("ticks")
blue = sns.light_palette("blue")[5]
red = sns.light_palette("red")[5]

### Load Data

Note: if the user wishes to train a model on different simulations, they should load the relevant training data below. 

In [ ]:
#Load surviving & disrupted subhalo features in matrix of size (#subhalos, #features)
m12i_surviving_properties = np.loadtxt('m12i_surviving_properties_dperiaaccvaccmaccaperi.txt')
m12i_destroyed_properties = np.loadtxt('m12i_destroyed_properties_dperiaaccvaccmaccaperi.txt')
m12i_surviving_labels = np.zeros(len(m12i_surviving_properties))
m12i_destroyed_labels = np.ones(len(m12i_destroyed_properties))

m12f_surviving_properties = np.loadtxt('m12f_surviving_properties_dperiaaccvaccmaccaperi.txt')
m12f_destroyed_properties = np.loadtxt('m12f_destroyed_properties_dperiaaccvaccmaccaperi.txt')
m12f_surviving_labels = np.zeros(len(m12f_surviving_properties))
m12f_destroyed_labels = np.ones(len(m12f_destroyed_properties))

In [ ]:
#Combine data from different hosts
subhalo_properties = np.concatenate((m12i_surviving_properties, \
                                     m12f_surviving_properties, \
                                     m12i_destroyed_properties, \
                                     m12f_destroyed_properties))

subhalo_labels = np.concatenate((m12i_surviving_labels, \
                                 m12f_surviving_labels, \
                                 m12i_destroyed_labels, \
                                 m12f_destroyed_labels))

In [ ]:
#Construct design matrix and target labels
X = subhalo_properties
y = subhalo_labels

#If only one feature is used, reshape design matrix
#X = X.reshape(-1,1)

In [ ]:
all_features = pd.DataFrame(X)
all_features = all_features.rename(index=str, columns={0:"$d_{peri}$", \
                                                       1: "$a_{acc}$", \
                                                       2: "$V_{acc}$", \
                                                       3: "$M_{acc}$", \
                                                       4: "$a_{peri}$"})

### Default Random Forest Classification

In [ ]:
rf = RandomForestClassifier(n_estimators=100, oob_score=True)
rf.fit(X, y)

In [ ]:
rf.feature_importances_

In [ ]:
rf.oob_score_

### Tune Random Forest Classifier with GridSearchCV

In [ ]:
#Hyperparameters to try:
parameters = {'n_estimators':(100,500,750,1000), "max_features": ["auto"],
              'criterion':["gini","entropy"], "min_samples_leaf": [1,2,4]}

# Train/test split:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25)

In [ ]:
# Do a grid search to find the highest n-fold cross-validation score:
n = 10
rf_tuned = GridSearchCV(rf, parameters, cv=n, verbose=1)
RFselector = rf_tuned.fit(X_train, y_train)

In [ ]:
# Print the best score and estimator:
print('Best OOB score:', RFselector.best_score_)
print(RFselector.best_estimator_)

#Print the best hyperparameters:
RFselector.best_params_

In [ ]:
#Raw accuracy
y_pred_all = RFselector.predict(X)
len(y_pred_all[y_pred_all==y])/len(y_pred_all)

In [ ]:
#Save finalized model
filename = 'finalized_rf.sav'
pickle.dump(RFselector, open(filename, 'wb'))

In [ ]:
#Load finalized model
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#Check raw accuracy of loaded model
len(loaded_model.predict(X)[loaded_model.predict(X)==y])/len(y_pred_all)

### Analyze Performance on Test Set

In [ ]:
y_pred = RFselector.predict(X_test)

In [ ]:
#Confusion matrix
cm = confusion_matrix(y_test, y_pred)
surv_frac = cm[0][0]/(cm[0][0]+cm[0][1])
dest_frac = cm[1][1]/(cm[1][0]+cm[1][1])

print('Fraction of surviving subhalos in test set labeled correctly: %0.2f' % surv_frac)
print('Fraction of disrupted subhalos in test set labeled correctly: %0.2f' % dest_frac)


plt.figure(figsize=(8,6))
plt.matshow(cm)
plt.title('Confusion Matrix', fontsize=18, position = (0.5,1.1))
plt.colorbar()
plt.ylabel('True label', fontsize=16)
plt.xlabel('Predicted label', fontsize=16, position = (0.5, -10.5))
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
#Check performance for different training/test sets
n_tests = 10
score = np.zeros(n_tests)
surv = np.zeros(n_tests)
dest = np.zeros(n_tests)

for i in range (0,n_tests):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.25)
    rf = RandomForestClassifier(n_estimators=1000, criterion='entropy', \
                                min_samples_leaf=1, oob_score=True)
    rf.fit(X_train, y_train)
    score[i] = rf.oob_score_
    y_pred = rf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    surv[i] = cm[0][0]/(cm[0][0]+cm[0][1])
    dest[i] = cm[1][1]/(cm[1][0]+cm[1][1])
    
print('Mean number of surviving subhalos identified correctly: %0.2f' % np.mean(surv))
print('Mean number of disrupted subhalos identified correctly: %0.2f' % np.mean(dest))

### ROC Curve 

In [ ]:
# Classify the test data and store classification probabilities:
BestRFselector = RFselector.best_estimator_
y_prob = BestRFselector.fit(X_train, y_train).predict_proba(X_test)

In [ ]:
# Compute ROC curve and area under curve (AUC) for each class:
labels = BestRFselector.classes_
fpr = dict()
tpr = dict()
roc_auc = dict()
for i,label in enumerate(labels):
    fpr[label], tpr[label], _ = roc_curve(y_test, y_prob[:, i], pos_label=label)
    roc_auc[label] = auc(fpr[label], tpr[label])

In [ ]:
roc_auc

In [ ]:
plt.figure(figsize=(8,6))
plt.plot([0, 1], [1, 1], color=red, linestyle='-', linewidth=3, label='Perfect Classifier (AUC = %0.2f)' % (1.0))
plt.plot(fpr[1], tpr[1], lw=3, label='Random Forest (AUC = %0.2f)' % (roc_auc[1]), color=blue)
plt.plot([0, 1], [0, 1], color='black', linestyle=':', linewidth=2.5, label='Random Classifier (AUC = %0.2f)' % (0.5))

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.025])
plt.tick_params(labelsize=16)
plt.xlabel('Destroyed Labeled Surviving', fontsize=20, labelpad=8)
plt.ylabel('Destroyed Labeled Destroyed', fontsize=20, labelpad=8)
plt.title('True vs. False Positive Classification Rate', fontsize=20)
plt.legend(loc="lower right", fontsize=16);